
# Creating Custom Phantoms for MC-GPU-PET

This supplementary notebook explains how to generate custom voxelized phantoms for MC-GPU simulations. 
While MC-GPU can read standard `penEasy` voxel formats, creating your own allows for modeling arbitrary geometries, lesions, or patient-specific distributions.

## The Voxel File Format (`.vox`)

The input file for MC-GPU expects a voxel geometry file defined by a header section followed by the raw data.

### 1. The Header
The header must strictly follow this structure:
```text
[SECTION VOXELS HEADER v.2008-04-13]
Nx Ny Nz   No. OF VOXELS IN X,Y,Z
Dx Dy Dz   VOXEL SIZE (cm) ALONG X,Y,Z
1          COLUMN NUMBER WHERE MATERIAL ID IS LOCATED
2          COLUMN NUMBER WHERE THE MASS DENSITY IS LOCATED
1          BLANK LINES AT END OF X,Y-CYCLES (1=YES,0=NO)
[END OF VXH SECTION]
```

### 2. The Data
The data follows the header. It is a list of voxels.
- **Order**: Z loops outer-most, then Y, then X.
- **Format**: `MaterialID Density Activity` (MC-GPU-PET extension: Activity is the 3rd column).
- **Blank Lines**: If "BLANK LINES" is 1 in the header, an empty line separates each X-row and double empty lines separate Z-slices.

Let's build a NEMA IEC Body Phantom from scratch using Python.


In [1]:

import numpy as np
import matplotlib.pyplot as plt
import math

# Configuration
FILENAME = "custom_nema_phantom.vox"
NVOX = [128, 128, 128]  # Dimensions
DVOX = [0.25, 0.25, 0.25] # Voxel size in cm



## Defining Materials and Activity

We define integer IDs for materials that match the material file list in our `MCGPU-PET.in` input file.
- **Material 1**: Air (Background)
- **Material 2**: Water (Phantom Body)

We also define activity levels (Becquerels) for different regions.


In [2]:

# Material IDs
MAT_AIR = 1
MAT_WATER = 2

# Densities (g/cm3)
DENS_AIR = 0.0012
DENS_WATER = 1.0

# Activity Levels (Bq)
ACT_AIR = 0.0
ACT_BKG = 50.0   # Background activity in the water cylinder
ACT_HOT = 200.0  # Hot spheres (4:1 ratio)
ACT_COLD = 0.0   # Cold spheres



## Defining Geometry

The NEMA phantom is effectively a large cylinder containing smaller spheres of various sizes.
We will calculate the position of every voxel and determine if it falls inside these shapes.


In [3]:

# Phantom Dimensions (cm)
CX = (NVOX[0] * DVOX[0]) / 2.0
CY = (NVOX[1] * DVOX[1]) / 2.0
CZ = (NVOX[2] * DVOX[2]) / 2.0

PHANTOM_RADIUS = 10.0 # 20cm diameter
SPHERE_RING_RADIUS = 5.72 

# Sphere Definitions
# 6 Spheres in a ring. 
# Typical NEMA: 10, 13, 17, 22, 28, 37 mm diameters
sphere_radii_cm = [d/20.0 for d in [10, 13, 17, 22, 28, 37]]
sphere_types = ['COLD', 'COLD', 'HOT', 'HOT', 'HOT', 'HOT']
sphere_centers = []

for i in range(6):
    angle_deg = i * 60.0
    angle_rad = math.radians(angle_deg)
    sx = CX + SPHERE_RING_RADIUS * math.cos(angle_rad)
    sy = CY + SPHERE_RING_RADIUS * math.sin(angle_rad)
    sz = CZ
    sphere_centers.append((sx, sy, sz))
    
print("Defined 6 spheres.")


Defined 6 spheres.



## Generating the Voxel File

We iterate through the grid (Z, Y, X) and determine the properties of each voxel.
Then we write the formatted string to the file.


In [4]:

def generate_voxel_file():
    print(f"Generating {FILENAME}...")
    with open(FILENAME, 'w') as f:
        # 1. Write Header
        f.write('[SECTION VOXELS HEADER v.2008-04-13]\n')
        f.write(f'{NVOX[0]} {NVOX[1]} {NVOX[2]}   No. OF VOXELS IN X,Y,Z\n')
        f.write(f'{DVOX[0]} {DVOX[1]} {DVOX[2]}   VOXEL SIZE (cm) ALONG X,Y,Z\n')
        f.write(' 1                  COLUMN NUMBER WHERE MATERIAL ID IS LOCATED\n')
        f.write(' 2                  COLUMN NUMBER WHERE THE MASS DENSITY IS LOCATED\n')
        f.write(' 1                  BLANK LINES AT END OF X,Y-CYCLES (1=YES,0=NO)\n')
        f.write('[END OF VXH SECTION]  # MCGPU-PET voxel format: Material  Density  Activity\n')

        # 2. Iterate Voxels (Z -> Y -> X)
        for k in range(NVOX[2]):
            z = (k + 0.5) * DVOX[2]
            for j in range(NVOX[1]):
                y = (j + 0.5) * DVOX[1]
                for i in range(NVOX[0]):
                    x = (i + 0.5) * DVOX[0]
                    
                    # Logic -------------------
                    dist_to_center_sq = (x - CX)**2 + (y - CY)**2
                    
                    # Default: Air
                    mat, dens, act = MAT_AIR, DENS_AIR, ACT_AIR
                    
                    # Inside Main Cylinder?
                    # (Restricting Z to avoid touching edges)
                    if dist_to_center_sq <= PHANTOM_RADIUS**2 and 2.0 < z < (NVOX[2]*DVOX[2] - 2.0):
                        mat, dens, act = MAT_WATER, DENS_WATER, ACT_BKG
                        
                        # Check Spheres
                        for s_idx, (sx, sy, sz) in enumerate(sphere_centers):
                            dist_to_sphere_sq = (x - sx)**2 + (y - sy)**2 + (z - sz)**2
                            if dist_to_sphere_sq <= sphere_radii_cm[s_idx]**2:
                                if sphere_types[s_idx] == 'HOT':
                                    act = ACT_HOT
                                else:
                                    act = ACT_COLD
                                break
                    # End Logic ----------------
                    
                    # Write Voxel: Mat Dens Act
                    f.write(f"{mat} {dens} {act}\n")
                
                f.write('\n') # End of X-row (Blank line)
            f.write('\n') # End of Z-slice (Double blank line)
            
    print("Generation Complete.")

generate_voxel_file()


Generating custom_nema_phantom.vox...
Generation Complete.



## Verification

We can verify the generated file by loading it back or running a simulation.
Now you can use `custom_nema_phantom.vox` in your `MCGPUWrapper` configuration:

```python
params = {
    ...
    'voxel_file': 'custom_nema_phantom.vox',
    ...
}
```

This approach allows you to programmatically create complex phantoms, such as:
- Importing patient CT data (converting HU to density/materials).
- creating mathematical phantoms (Derenzo, XCAT).
- Adding complex lesions or motion artifacts.
